# Progetto


Prendere in input un file <code>FASTQ</code> di reads di **lunghezza costante**, una **soglia di frequenza** <code>F</code> (tra 0 e 1) e una **lunghezza** <code>k</code> e produrre in output il report dell'uso dei ***k-mers*** per posizione nei reads in input. Un ***k-mer*** è una ***sottostringa di read*** di lunghezza **k**. Il **report** (possibilmente grafico) deve presentare per ciascun ***k-mer*** che ha una frequenza almeno pari a **F** (nel dataset di input) quante volte appare in ciascuna delle posizioni dei reads dalla prima a <code>L - k + 1</code> (**L**: lunghezza dei reads). Selezionare il ***k-mer*** che appare più volte in una certa posizione e produrre in un file <code>FASTA</code> i reads che lo contengono (in quella posizione)  con la loro **qualità media**. Usare <code>Biopython</code> ogni volta che è possibile.

# Import librerie

In [ ]:
from Bio import SeqIO
import re

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Funzione di stampa per percentuali

In [ ]:
APPROX = 2
perc = True

def printable_percentage(f, approx, mult = True):
    if mult:
        f *= 100
    f = np.round(f, approx)
    if mult:
        f = str(f) + "%"
    return f

# Funzione di trasformazione da *dict* a *np.array*

In [ ]:
def dict_to_array(freq_dict, size, freq_type = np.int32):
    freq_array = np.zeros(size, freq_type)
    for key, value in freq_dict.items():
        freq_array[key] = value
        
    return freq_array

# Parametri di input

In [ ]:
iPath = "./SRR18961685-5000.fastq"
oPath = "readSelected.fasta"

F = 0.001
k = 11

assert 0 <= F <= 1, f"Frequence {F} out of range [0, 1]"

# Lettura del file ***fastq*** con ***Biopython***

In [ ]:
fastq_content = SeqIO.parse(iPath, "fastq")

In [ ]:
fastq_content

In [ ]:
indexed_content = list(fastq_content)
n_read = len(indexed_content)
read_length = int(re.search(r'length=(\d+)', indexed_content[0].description).group(1))

assert read_length >= k, f"k={k} > L={read_length}"

***L***: Lunghezza dei read (costante)

In [ ]:
read_length

***N***: Numero dei read

In [ ]:
n_read

# Costruzione del dizionario dei k-meri

Si costruisce un **dizionario** che ha:

* per **chiave** i *k-meri* estratti dai read
* per **valore** una *lista* di due elementi:
  * un dizionario <strong>d</strong> che ha 
    * come chiavi ***key*** le posizioni di match del k-mero;
    * come valori ***d[key]*** il numero di match del k-mero in quella posizione
  * la **somma** di tutti i match del k-mero
        
        

<code>kmers_pos_freq : k-mero -> [d[i], tot_match]</code>

<code>d[i] = numero di match del k-mero nella posizione i di ogni read</code>
        
        

In [ ]:
kmers_pos_freq = {}

for record in indexed_content:
    
    seq_string = str(record.seq)
    assert (len(seq_string) == read_length), f"Error in {iPath} file: different lengths of records"
    
    for i in range(len(seq_string) - k + 1):
        
        kmer = seq_string[i:i + k]
        
        if kmer in kmers_pos_freq:
            kmers_pos_freq[kmer][0][i] = kmers_pos_freq[kmer][0].get(i, 0) + 1
            kmers_pos_freq[kmer][1] += 1
        else:
            kmers_pos_freq[kmer] = [{i:1}, 1]

In [ ]:
kmers_pos_freq

**Numero dei k-meri distinti** : lunghezza del dizionario dei k-meri

In [ ]:
n_kmers = len(kmers_pos_freq)

In [ ]:
n_kmers

**Numero totale dei k-meri (non necessariamente distinti)**: <code>N * (L - k + 1)</code>

In [ ]:
n_kmers_appearence = 0
for value in kmers_pos_freq.values():
    n_kmers_appearence += value[1]

assert n_kmers_appearence == (n_read * (read_length - k + 1))

In [ ]:
n_kmers_appearence

## Selezione dei k-meri di frequenza minima

Vengono selezionati solo i **k-meri** di frequenza maggiore o uguale <code>F</code>, dove la frequenza di un k-mero è il suo numero di apparizioni diviso il numero totale di apparizioni di tutti i k-meri.

<code>kmers_pos_freq[kmer] / n_kmers_appearence >= F </code>

Questa volta i **match per posizione** di ogni **k-mero** vengono memorizzate in un ***numpy-array*** <code>a</code>, dove <code>a[i] = numero di apparizioni del k-mero in posizione i</code>.

In [ ]:
F = 0.0011

In [ ]:
kmers_threshold_freq = {}

for key, value in kmers_pos_freq.items():
    if value[1]/n_kmers_appearence >= F:
        kmers_threshold_freq[key] = [dict_to_array(value[0], read_length - k + 1), value[1]]

In [ ]:
kmers_threshold_freq

**Numero dei k-meri distinti sopra la soglia minima di frequenza**

In [ ]:
n_kmers_threshold = len(kmers_threshold_freq)

In [ ]:
n_kmers_threshold

**Numero totale di apparizioni dei k-meri sopra la soglia minima di frequenza**

In [ ]:
n_kmers_treshold_appearence = 0
for value in kmers_threshold_freq.values():
    n_kmers_treshold_appearence += value[1]

In [ ]:
n_kmers_treshold_appearence

# Analisi e plotting dei risultati

Per ogni **k-mero** selezionato (o al massimo <code>maxBars</code> k-meri) viene riportato il grafico a barre che mostra **il numero di apparizioni per posizione**. Le posizioni sono **0-indexed**.

In [ ]:
columns = 2
maxRows = 10

maxBars = columns * maxRows

In [ ]:
x_values = np.empty(read_length - k + 1)

for i in range(read_length - k + 1):
    x_values[i] = i

In [ ]:
i = 0
length = columns

for kmer in kmers_threshold_freq.keys():
    
    if (i % columns) == 0:
        if i < n_kmers_threshold - columns + 1:
            fig, axs = plt.subplots(1, columns)
        else:
            fig, axs = plt.subplots(1, n_kmers_threshold - i)
            length = n_kmers_threshold - i
        
        fig.tight_layout()
        
    if length > 1:
        axs[i%columns].set_title(kmer)
        axs[i%columns].bar(x_values, kmers_threshold_freq[kmer][0])
        axs[i%columns].set(xlabel='Position', ylabel='Frequence')
    else:
        axs.set_title(kmer)
        axs.bar(x_values, kmers_threshold_freq[kmer][0])
        axs.set(xlabel='Position', ylabel='Frequence')
        
    i = i + 1
    if(i == maxBars):
        break
        

## Ricerca dei k-meri sopra la frequenza minima

Ricerca per **prefisso** dei **k-meri** che appaiono sopra la **frequenza minima** <code>F</code>. Le percentuali si riferiscono solo ai **k-meri** sopra la **soglia minima di frequenza**.

In [ ]:
prefix = r"A"

In [ ]:
print(f"{k}-mers starting with prefix '{prefix}':")
print()
matches = [match for match in list(kmers_threshold_freq.keys()) if re.match(prefix, match, re.IGNORECASE)]
matches.sort(reverse=True, key = lambda x : kmers_threshold_freq[x][1])

totAppearences = 0

for kmer in matches:
    print(f"{kmer}: {kmers_threshold_freq[kmer][1]} appearences ({printable_percentage(kmers_threshold_freq[kmer][1]/n_kmers_treshold_appearence, APPROX, perc)})")
    totAppearences += kmers_threshold_freq[kmer][1]
    
print()
print(f"Total: {totAppearences} appearences on {n_kmers_treshold_appearence} ({printable_percentage(totAppearences/n_kmers_treshold_appearence, APPROX, perc)})")

## Analisi dei singoli k-meri

**Analisi statistiche** dei dati per singoli **k-meri**.

In [ ]:
kmer = "AGGGCAGAGGG"

In [ ]:
freq_array = kmers_threshold_freq.get(kmer, None)

if(freq_array != None):
    
    print(f"K-mer: {kmer}")
    print()
    
    print(f"Total: {freq_array[1]} appearences on {n_kmers_appearence} total k-mers ({printable_percentage(freq_array[1]/n_kmers_appearence, APPROX, perc)})")
    
    print()
    print(f"Mean: {np.round(np.mean(freq_array[0]), APPROX)} appearences per position")
    tmp = np.argmax(freq_array[0])
    print(f"Max: position {tmp} with {freq_array[0][tmp]} appearences ({printable_percentage(freq_array[0][tmp]/freq_array[1], APPROX, perc)})")
    print(f"Standard deviation: {np.round(np.std(freq_array[0]), APPROX)}")
    print()
    
    print(f"Quantile 1: {np.round(np.percentile(freq_array[0], 25), APPROX)}")
    print(f"Median: {np.round(np.median(freq_array[0]), APPROX)}")
    print(f"Quantile 3: {np.round(np.percentile(freq_array[0], 75), APPROX)}")
    
    plt.bar(x_values, freq_array[0])
    plt.ylabel('Frequence')
    plt.xlabel('Position')
    plt.title(kmer)
    plt.show()
            
else:
    print(kmer, "not found")

**Ricerca numero di apparizioni** per intervallo:

In [ ]:
start = 13
end = 13

assert freq_array != None, f"{kmer}: k-mer not found"
assert 0 <= start <= read_length - k, f"Start position {start} out of range for [0, {read_length - k}]"
assert 0 <= end <= read_length - k, f"End position {end} out of range for [0, {read_length - k}]"
assert start <= end, f"start_position={start} > end_position={end}"

tmp = 0
for pos in range(start, end + 1):
    tmp += freq_array[0][pos]

if(start == end):
    print(f"Position {start}: {tmp} appearence on {freq_array[1]} ({printable_percentage(tmp/freq_array[1], APPROX, perc)})")
else:
    print(f"Interval [{start}, {end}]: {tmp} appearence on {freq_array[1]} ({printable_percentage(tmp/freq_array[1], APPROX, perc)})")

# Ricerca k-mero/i con più apparizioni in una certa posizione

### Posizione scelta

In [ ]:
position = 137

assert 0 <= position <= read_length - k, f"Position {position} out of range for interval [0, {read_length - k}]"

### Lista di tutti i k-meri che appaiono nella posizione scelta

<code>appearence_in_pos[i] = [(kmer, frequence of kmer in position i)]

In [ ]:
appearence_in_pos = [(kmer, kmers_threshold_freq[kmer][0][position]) for kmer in kmers_threshold_freq.keys() if kmers_threshold_freq[kmer][0][position] > 0]

In [ ]:
appearence_in_pos

**Numero massimo di apparizioni** nella posizione scelta:

In [ ]:
assert len(appearence_in_pos) > 0, "Empty list"

max_appearence = max(appearence_in_pos, key = lambda x : x[1])[1]

In [ ]:
max_appearence

### Lista dei k-meri nella posizione scelta con più apparizioni 

In [ ]:
max_kmers_in_pos = [pair[0] for pair in appearence_in_pos if pair[1] == max_appearence]

In [ ]:
max_kmers_in_pos

### Selezione dei k-meri da ricercare nei reads

Attraverso una **maschera binaria** si scelgono i k-meri con cui si andranno a selezionare i **reads** (nel caso ce ne sia più di uno). La maschera deve avere la **stessa lunghezza** di <code>max_kmers_in_pos</code>.

In [ ]:
mask = (True,)

assert len(mask) == len(max_kmers_in_pos), "mask length different from list length"

In [ ]:
kmers_in_pos = [max_kmers_in_pos[i] for i in range(len(max_kmers_in_pos)) if mask[i]]

In [ ]:
kmers_in_pos

# Selezione dei reads che contengono i k-meri nella posizione scelta

### Selezione dei reads

Salvataggio dei **reads selezionati** nella lista <code>read_selected</code>.

In [ ]:
read_selected = []

for record in indexed_content:
    seq_string = str(record.seq)
    
    for kmer in kmers_in_pos:
        
        if seq_string[position : position + k] == kmer:
            read_selected.append(record)
            break

In [ ]:
print(f"N° reads selected: {len(read_selected)}")

# Calcolo delle qualità medie per ogni read selezionato

La classe **SeqRecord** permette di accedere alle qualità di ogni di base di un read tramite l'attributo <code>letter_annotations</code>.

In [ ]:
assert len(read_selected) > 0, "0 reads selected"

read_selected[0].letter_annotations

**Chiave** del dizionario che raccoglie le qualità:

In [ ]:
key = str(list(read_selected[0].letter_annotations.keys())[0])

In [ ]:
key

Lista delle **qualità medie** per ogni read selezionato:

In [ ]:
indexed_mean_qual = [np.mean(record.letter_annotations[key]) for record in read_selected]

In [ ]:
indexed_mean_qual

Estrazione dell'**identificatore di sequenza**

In [ ]:
reads_id = re.search(r'(\w+)\.', read_selected[0].id).group(1)

In [ ]:
reads_id

# Plotting e analisi delle qualità medie

Ogni **read** viene enumerato con un indice in <code>[0, len(indexed_mean_qual) - 1]</code>.

In [ ]:
print(f"Mean of mean qualities: {np.round(np.mean(indexed_mean_qual), APPROX)}")
      
tmp = np.argmax(indexed_mean_qual)
print(f"Max: {np.round(indexed_mean_qual[tmp], APPROX)} (index {tmp})")
      
tmp = np.argmin(indexed_mean_qual)
print(f"Min: {np.round(indexed_mean_qual[tmp], APPROX)} (index {tmp})")
      
print(f"Standard deviation: {np.round(np.std(indexed_mean_qual), APPROX)}")

plt.bar(range(len(indexed_mean_qual)), indexed_mean_qual)
plt.ylabel('Mean quality')
plt.xlabel('Index')
plt.title(reads_id)
plt.show()

In [ ]:
index = 11
assert 0 <= index < len(indexed_mean_qual), f"index out of range for [0, {len(indexed_mean_qual) - 1}]"

print("Index: ", index)
print()
print("Mean quality:", np.round(indexed_mean_qual[index], APPROX))
print(f"Read: '{read_selected[index].seq}'")

# Scrittura del file *fasta* con le qualità medie

Aggiunta del dato riguardante le **qualità medie** nel commento di ogni **record**:

In [ ]:
for i in range(len(read_selected)):
    read_selected[i].description += f" mean_quality={indexed_mean_qual[i]}"

In [ ]:
read_selected[0].description

**Scrittura** del **file**:

In [ ]:
SeqIO.write(read_selected, oPath, "fasta")